In [1]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=a49d45880271218993c3bf0b9ea79c15c3bf0047bc924b2a9c976285af713615
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
# Import Sparksession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Data_Wrangling").getOrCreate()

In [3]:
# Print PySpark and Python versions
import sys
print('Python version: '+sys.version)
print('Spark version: '+spark.version)

Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Spark version: 3.5.1


In [4]:
# Read data
file_location = "movie_data_part1.csv"
file_type = "csv"
infer_schema = "false"
first_row_is_header = "true"
delimiter = "|"

df = spark.read.format(file_type)\
.option("inferSchema", infer_schema)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load(file_location)

In [5]:
# Print Metadata
df.printSchema()

root
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: string (nullable = true)



In [6]:
# Print Metadata - 2nd method
df.dtypes

[('belongs_to_collection', 'string'),
 ('budget', 'string'),
 ('id', 'string'),
 ('original_language', 'string'),
 ('original_title', 'string'),
 ('overview', 'string'),
 ('popularity', 'string'),
 ('production_companies', 'string'),
 ('production_countries', 'string'),
 ('release_date', 'string'),
 ('revenue', 'string'),
 ('runtime', 'string'),
 ('status', 'string'),
 ('tagline', 'string'),
 ('title', 'string'),
 ('vote_average', 'string')]

In [7]:
#  Count data
df.count()
print('The total number of records in the movie dataset are '+str(df.count()))

The total number of records in the movie dataset are 43998


In [8]:
#  Drop Columns
drop_columns_list=["overview","belongs_to_collection","production_companies","production_countries","status","original_title"]
df1=df.drop(*drop_columns_list)

In [9]:
# Subsetting Columns
select_columns=['id','budget','popularity','release_date','revenue','title']
df=df.select(*select_columns)
df.show()

+-----+-------+------------------+------------+-------+--------------------+
|   id| budget|        popularity|release_date|revenue|               title|
+-----+-------+------------------+------------+-------+--------------------+
|43000|      0|             2.503|  1962-05-23|      0|The Elusive Corporal|
|43001|      0|              5.51|  1962-11-12|      0|  Sundays and Cybele|
|43002|      0|              5.62|  1962-05-24|      0|Lonely Are the Brave|
|43003|      0|             7.159|  1975-03-12|      0|          F for Fake|
|43004| 500000|             3.988|  1962-10-09|      0|Long Day's Journe...|
|43006|      0|             3.194|  1962-03-09|      0|           My Geisha|
|43007|      0|             2.689|  1962-10-31|      0|Period of Adjustment|
|43008|      0|             6.537|  1959-03-13|      0|    The Hanging Tree|
|43010|      0|             4.297|  1962-01-01|      0|Sherlock Holmes a...|
|43011|      0|             4.417|  1962-01-01|      0|  Sodom and Gomorrah|

In [10]:
# Subsetting Columns
df = spark.read.format(file_type)\
.option("inferSchema", infer_schema)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load(file_location)
df=df.select(df[2],df[1],df[6],df[9],df[10],df[14])
df.show()

+-----+-------+------------------+------------+-------+--------------------+
|   id| budget|        popularity|release_date|revenue|               title|
+-----+-------+------------------+------------+-------+--------------------+
|43000|      0|             2.503|  1962-05-23|      0|The Elusive Corporal|
|43001|      0|              5.51|  1962-11-12|      0|  Sundays and Cybele|
|43002|      0|              5.62|  1962-05-24|      0|Lonely Are the Brave|
|43003|      0|             7.159|  1975-03-12|      0|          F for Fake|
|43004| 500000|             3.988|  1962-10-09|      0|Long Day's Journe...|
|43006|      0|             3.194|  1962-03-09|      0|           My Geisha|
|43007|      0|             2.689|  1962-10-31|      0|Period of Adjustment|
|43008|      0|             6.537|  1959-03-13|      0|    The Hanging Tree|
|43010|      0|             4.297|  1962-01-01|      0|Sherlock Holmes a...|
|43011|      0|             4.417|  1962-01-01|      0|  Sodom and Gomorrah|

In [11]:
# Identifying missing values
from pyspark.sql.functions import *
df.filter((df['popularity']=='')|df['popularity'].isNull()|isnan(df['popularity'])).count()

215

In [12]:
# Identifying missing values
df.select([count(when((col(c)=='') | col(c).isNull() |isnan(c), c)).alias(c) for c in df.columns]).show()

+---+------+----------+------------+-------+-----+
| id|budget|popularity|release_date|revenue|title|
+---+------+----------+------------+-------+-----+
|125|   125|       215|         221|    215|  304|
+---+------+----------+------------+-------+-----+



In [13]:
# One way Frequencies
df.groupBy(df['title']).count().show()

+--------------------+-----+
|               title|count|
+--------------------+-----+
|   The Corn Is Green|    1|
|Meet The Browns -...|    1|
|Morenita, El Esca...|    1|
| Father Takes a Wife|    1|
|The Werewolf of W...|    1|
|My Wife Is a Gang...|    1|
|Depeche Mode: Tou...|    1|
|  A Woman Is a Woman|    1|
|History Is Made a...|    1|
|      Colombian Love|    1|
|        Ace Attorney|    1|
|     Not Like Others|    1|
|40 Guns to Apache...|    1|
|          Middle Men|    1|
|         It's a Gift|    1|
|    La Vie de Bohème|    1|
|Rasputin: The Mad...|    1|
|The Ballad of Jac...|    1|
|         How to Deal|    1|
|             Freaked|    1|
+--------------------+-----+
only showing top 20 rows



In [14]:
#Frequencies - descending order
df.groupby(df['title']).count().sort(desc("count")).show(10, False)

+--------------------+-----+
|title               |count|
+--------------------+-----+
|NULL                |304  |
|Les Misérables      |8    |
|The Three Musketeers|8    |
|Cinderella          |8    |
|A Christmas Carol   |7    |
|The Island          |7    |
|Dracula             |7    |
|Hamlet              |7    |
|Frankenstein        |7    |
|Cleopatra           |6    |
+--------------------+-----+
only showing top 10 rows



In [15]:
#  One way frquencies with filters
df_temp=df.filter((df['title']!='')&(df['title'].isNotNull()) & (~isnan(df['title'])))
df_temp.groupby(df_temp['title']).count().filter("`count` >4").sort(col("count").desc()).show(10,False)
df_temp.groupby(df_temp['title']).count().filter("`count` >4").sort(col("count").desc()).count()

+--------------------+-----+
|title               |count|
+--------------------+-----+
|Les Misérables      |8    |
|The Three Musketeers|8    |
|Cinderella          |8    |
|A Christmas Carol   |7    |
|The Island          |7    |
|Frankenstein        |7    |
|Dracula             |7    |
|Hamlet              |7    |
|Treasure Island     |6    |
|The Lost World      |6    |
+--------------------+-----+
only showing top 10 rows



43

In [16]:
# Casting
df = df.withColumn('budget',df['budget'].cast("float"))

In [17]:
#Before Casting
df.dtypes

[('id', 'string'),
 ('budget', 'float'),
 ('popularity', 'string'),
 ('release_date', 'string'),
 ('revenue', 'string'),
 ('title', 'string')]

In [18]:
#After Casting
df.dtypes


[('id', 'string'),
 ('budget', 'float'),
 ('popularity', 'string'),
 ('release_date', 'string'),
 ('revenue', 'string'),
 ('title', 'string')]

In [19]:
# Casting multiple variables
from pyspark.sql.types import *

#Identifying and assiging lists of variables
int_vars=['id']
float_vars=['budget', 'popularity', 'revenue']
date_vars=['release_date']

for column in int_vars:
	df=df.withColumn(column,df[column].cast(IntegerType()))
for column in float_vars:
	df=df.withColumn(column,df[column].cast(FloatType()))
for column in date_vars:
	df=df.withColumn(column,df[column].cast(DateType()))

In [20]:
#After Casting
df.dtypes

[('id', 'int'),
 ('budget', 'float'),
 ('popularity', 'float'),
 ('release_date', 'date'),
 ('revenue', 'float'),
 ('title', 'string')]

In [21]:
#After Casting output
df.show(10,False)

+-----+--------+----------+------------+-------+---------------------------------------+
|id   |budget  |popularity|release_date|revenue|title                                  |
+-----+--------+----------+------------+-------+---------------------------------------+
|43000|0.0     |2.503     |1962-05-23  |0.0    |The Elusive Corporal                   |
|43001|0.0     |5.51      |1962-11-12  |0.0    |Sundays and Cybele                     |
|43002|0.0     |5.62      |1962-05-24  |0.0    |Lonely Are the Brave                   |
|43003|0.0     |7.159     |1975-03-12  |0.0    |F for Fake                             |
|43004|500000.0|3.988     |1962-10-09  |0.0    |Long Day's Journey Into Night          |
|43006|0.0     |3.194     |1962-03-09  |0.0    |My Geisha                              |
|43007|0.0     |2.689     |1962-10-31  |0.0    |Period of Adjustment                   |
|43008|0.0     |6.537     |1959-03-13  |0.0    |The Hanging Tree                       |
|43010|0.0     |4.297

In [22]:
# Describe function
df.describe().show()

+-------+------------------+--------------------+------------------+-------------------+--------------------+
|summary|                id|              budget|        popularity|            revenue|               title|
+-------+------------------+--------------------+------------------+-------------------+--------------------+
|  count|             43784|               43873|             43783|              43783|               43694|
|   mean|44502.304312077475|  3736901.8349631676| 5.295444259579189|  9697079.597382545|            Infinity|
| stddev|  27189.6465886264|1.5871814952777285E7|6.1680305192082665|5.687938449628816E7|                 NaN|
|    min|                 2|                 0.0|               0.6|                0.0|!Women Art Revolu...|
|    max|            100988|               3.8E8|             180.0|       2.78796518E9|       시크릿 Secret|
+-------+------------------+--------------------+------------------+-------------------+--------------------+



In [23]:
# Median Calculation
df_temp = df.filter((df['budget']!=0)&(df['budget'].isNotNull()) & (~isnan(df['budget'])))
median=df_temp.approxQuantile('budget',[0.5],0.001)
print ('The median of budget is '+str(median))

The median of budget is [7000000.0]


In [24]:
# Distinct Counts
df.agg(countDistinct(col("title")).alias("count")).show()

+-----+
|count|
+-----+
|41138|
+-----+



In [25]:
# Distinct Values
df.select('title').distinct().show(10,False)

+---------------------------------------------+
|title                                        |
+---------------------------------------------+
|The Corn Is Green                            |
|Meet The Browns - The Play                   |
|Morenita, El Escandalo                       |
|Father Takes a Wife                          |
|The Werewolf of Washington                   |
|My Wife Is a Gangster                        |
|Depeche Mode: Touring the Angel Live in Milan|
|A Woman Is a Woman                           |
|History Is Made at Night                     |
|Colombian Love                               |
+---------------------------------------------+
only showing top 10 rows



In [26]:
# Distinct Aggregations
df_temp=df.withColumn('release_year',year('release_date'))
df_temp.groupBy("release_year").agg(countDistinct("title")).show(10,False)

+------------+---------------------+
|release_year|count(DISTINCT title)|
+------------+---------------------+
|1959        |271                  |
|1990        |496                  |
|1975        |365                  |
|1977        |415                  |
|1924        |19                   |
|2003        |1199                 |
|2007        |1896                 |
|2018        |4                    |
|1974        |434                  |
|2015        |13                   |
+------------+---------------------+
only showing top 10 rows



In [27]:
# Datetime extractions
df_temp=df_temp.withColumn('release_month',month('release_date'))
df_temp=df_temp.withColumn('release_day',dayofmonth('release_date'))


In [28]:
# Output of datetime extractions
df_temp.show()

+-----+---------+----------+------------+---------+--------------------+------------+-------------+-----------+
|   id|   budget|popularity|release_date|  revenue|               title|release_year|release_month|release_day|
+-----+---------+----------+------------+---------+--------------------+------------+-------------+-----------+
|43000|      0.0|     2.503|  1962-05-23|      0.0|The Elusive Corporal|        1962|            5|         23|
|43001|      0.0|      5.51|  1962-11-12|      0.0|  Sundays and Cybele|        1962|           11|         12|
|43002|      0.0|      5.62|  1962-05-24|      0.0|Lonely Are the Brave|        1962|            5|         24|
|43003|      0.0|     7.159|  1975-03-12|      0.0|          F for Fake|        1975|            3|         12|
|43004| 500000.0|     3.988|  1962-10-09|      0.0|Long Day's Journe...|        1962|           10|          9|
|43006|      0.0|     3.194|  1962-03-09|      0.0|           My Geisha|        1962|            3|     

In [29]:
# Filtering based on like
df.filter(df['title'].like('Meet%')).show(10,False)

+-----+---------+----------+------------+-----------+--------------------------+
|id   |budget   |popularity|release_date|revenue    |title                     |
+-----+---------+----------+------------+-----------+--------------------------+
|43957|500000.0 |2.649     |2005-06-28  |1000000.0  |Meet The Browns - The Play|
|39997|0.0      |3.585     |1989-11-15  |0.0        |Meet the Hollowheads      |
|16710|0.0      |11.495    |2008-03-21  |4.1939392E7|Meet the Browns           |
|20430|0.0      |3.614     |2004-01-29  |0.0        |Meet Market               |
|76435|0.0      |1.775     |2011-03-31  |0.0        |Meet the In-Laws          |
|76516|5000000.0|4.05      |1990-11-08  |485772.0   |Meet the Applegates       |
|7278 |3.0E7    |11.116    |2008-01-24  |8.4646832E7|Meet the Spartans         |
|32574|0.0      |7.42      |1941-03-14  |0.0        |Meet John Doe             |
|40506|0.0      |4.814     |1997-01-31  |0.0        |Meet Wally Sparks         |
|40688|2.4E7    |6.848     |

In [30]:
# Filtering based on not like
df.filter(~df['title'].like('%s')).show(10,False)

+-----+--------+----------+------------+-------+---------------------------------------+
|id   |budget  |popularity|release_date|revenue|title                                  |
+-----+--------+----------+------------+-------+---------------------------------------+
|43000|0.0     |2.503     |1962-05-23  |0.0    |The Elusive Corporal                   |
|43001|0.0     |5.51      |1962-11-12  |0.0    |Sundays and Cybele                     |
|43002|0.0     |5.62      |1962-05-24  |0.0    |Lonely Are the Brave                   |
|43003|0.0     |7.159     |1975-03-12  |0.0    |F for Fake                             |
|43004|500000.0|3.988     |1962-10-09  |0.0    |Long Day's Journey Into Night          |
|43006|0.0     |3.194     |1962-03-09  |0.0    |My Geisha                              |
|43007|0.0     |2.689     |1962-10-31  |0.0    |Period of Adjustment                   |
|43008|0.0     |6.537     |1959-03-13  |0.0    |The Hanging Tree                       |
|43010|0.0     |4.297

In [31]:
# Filtering based on not regular expressions - P1
df.filter(df['title'].rlike('[A-Z]*ove')).show(10,False)

+-----+------+----------+------------+------------+------------------------+
|id   |budget|popularity|release_date|revenue     |title                   |
+-----+------+----------+------------+------------+------------------------+
|43100|0.0   |7.252     |1959-10-07  |0.0         |General Della Rovere    |
|43152|0.0   |5.126     |2001-06-21  |0.0         |Love on a Diet          |
|43191|0.0   |4.921     |1952-08-29  |0.0         |Beware, My Lovely       |
|43281|0.0   |2.411     |1989-11-22  |0.0         |Love Without Pity       |
|43343|0.0   |3.174     |1953-12-25  |0.0         |Easy to Love            |
|43347|3.0E7 |14.863    |2010-11-22  |1.02820008E8|Love & Other Drugs      |
|43362|0.0   |1.705     |1952-02-23  |0.0         |Love Is Better Than Ever|
|43363|0.0   |2.02      |1952-05-29  |0.0         |Lovely to Look At       |
|43395|0.0   |4.758     |1950-11-10  |0.0         |Two Weeks with Love     |
|43455|0.0   |4.669     |1948-08-23  |0.0         |The Loves of Carmen     |

In [32]:
# Filtering based on contain function
df.filter(df.title.contains('ove')).show()

+-----+------+----------+------------+------------+--------------------+
|   id|budget|popularity|release_date|     revenue|               title|
+-----+------+----------+------------+------------+--------------------+
|43100|   0.0|     7.252|  1959-10-07|         0.0|General Della Rovere|
|43152|   0.0|     5.126|  2001-06-21|         0.0|      Love on a Diet|
|43191|   0.0|     4.921|  1952-08-29|         0.0|   Beware, My Lovely|
|43281|   0.0|     2.411|  1989-11-22|         0.0|   Love Without Pity|
|43343|   0.0|     3.174|  1953-12-25|         0.0|        Easy to Love|
|43347| 3.0E7|    14.863|  2010-11-22|1.02820008E8|  Love & Other Drugs|
|43362|   0.0|     1.705|  1952-02-23|         0.0|Love Is Better Th...|
|43363|   0.0|      2.02|  1952-05-29|         0.0|   Lovely to Look At|
|43395|   0.0|     4.758|  1950-11-10|         0.0| Two Weeks with Love|
|43455|   0.0|     4.669|  1948-08-23|         0.0| The Loves of Carmen|
|43483|   0.0|     3.161|  1946-12-25|         0.0|

In [33]:
# Filtering based on regular expression - P2
df.filter(df['title'].rlike('\w*ove')).count()

1025

In [34]:
# Filtering based on regular expression with ColRegex to subset columns - P1
df.select(df.colRegex("`(re)+?.+`")).printSchema()

root
 |-- release_date: date (nullable = true)
 |-- revenue: float (nullable = true)



In [35]:
# Filtering based on regular expression with ColRegex to subset columns - P2
df.select(df.colRegex("`re\w*`")).printSchema()

root
 |-- release_date: date (nullable = true)
 |-- revenue: float (nullable = true)



In [36]:
# Filtering based on regular expression with ColRegex to subset columns - P3
df.select(df.colRegex("`\w*e`")).printSchema()

root
 |-- release_date: date (nullable = true)
 |-- revenue: float (nullable = true)
 |-- title: string (nullable = true)



In [37]:
# New Columns
mean_pop=df.agg({'popularity': 'mean'}).collect()[0]['avg(popularity)']
count_obs= df.count()
df=df.withColumn('mean_popularity',lit(mean_pop))
df=df.withColumn('varaiance',pow((df['popularity']-df['mean_popularity']),2))
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- budget: float (nullable = true)
 |-- popularity: float (nullable = true)
 |-- release_date: date (nullable = true)
 |-- revenue: float (nullable = true)
 |-- title: string (nullable = true)
 |-- mean_popularity: double (nullable = false)
 |-- varaiance: double (nullable = true)



In [38]:
# New Columns - output
df.show()

+-----+---------+----------+------------+---------+--------------------+-----------------+--------------------+
|   id|   budget|popularity|release_date|  revenue|               title|  mean_popularity|           varaiance|
+-----+---------+----------+------------+---------+--------------------+-----------------+--------------------+
|43000|      0.0|     2.503|  1962-05-23|      0.0|The Elusive Corporal|5.295444259579189|   7.797744825681142|
|43001|      0.0|      5.51|  1962-11-12|      0.0|  Sundays and Cybele|5.295444259579189|0.046034263963398346|
|43002|      0.0|      5.62|  1962-05-24|      0.0|Lonely Are the Brave|5.295444259579189| 0.10533635435520017|
|43003|      0.0|     7.159|  1975-03-12|      0.0|          F for Fake|5.295444259579189|  3.4728396990815216|
|43004| 500000.0|     3.988|  1962-10-09|      0.0|Long Day's Journe...|5.295444259579189|   1.709410711356759|
|43006|      0.0|     3.194|  1962-03-09|      0.0|           My Geisha|5.295444259579189|   4.416067952

In [39]:
# variance calculation
variance_sum=df.agg({'varaiance': 'sum'}).collect()[0]['sum(varaiance)']
print(variance_sum)
variance_population= variance_sum/(count_obs-1)
print(variance_population)

1665668.6984730042
37.858688057663116


In [40]:
# Standard deviation
import math
math.sqrt(variance_population)

6.152941415100839

In [41]:
# Variance calculation sub output
df.show()

+-----+---------+----------+------------+---------+--------------------+-----------------+--------------------+
|   id|   budget|popularity|release_date|  revenue|               title|  mean_popularity|           varaiance|
+-----+---------+----------+------------+---------+--------------------+-----------------+--------------------+
|43000|      0.0|     2.503|  1962-05-23|      0.0|The Elusive Corporal|5.295444259579189|   7.797744825681142|
|43001|      0.0|      5.51|  1962-11-12|      0.0|  Sundays and Cybele|5.295444259579189|0.046034263963398346|
|43002|      0.0|      5.62|  1962-05-24|      0.0|Lonely Are the Brave|5.295444259579189| 0.10533635435520017|
|43003|      0.0|     7.159|  1975-03-12|      0.0|          F for Fake|5.295444259579189|  3.4728396990815216|
|43004| 500000.0|     3.988|  1962-10-09|      0.0|Long Day's Journe...|5.295444259579189|   1.709410711356759|
|43006|      0.0|     3.194|  1962-03-09|      0.0|           My Geisha|5.295444259579189|   4.416067952

In [42]:
#  New Columns - Method 2
def new_cols(budget,popularity):
 if budget<10000000: budget_cat='Small'
 elif budget<100000000: budget_cat='Medium'
 else: budget_cat='Big'
 if popularity<3: ratings='Low'
 elif popularity<5: ratings='Mid'
 else: ratings='High'
 return budget_cat,ratings


# Apply the user defined function on the dataframe
udfB=udf(new_cols,StructType([StructField("budget_cat", StringType(), True),StructField("ratings", StringType(), True)]))
temp_df=df.select('id','budget','popularity').withColumn("newcat",udfB("budget","popularity"))
# Unbundle the struct type columns into individual columns and drop the struct type
df_with_newcols = temp_df.select('id','budget','popularity','newcat').withColumn('budget_cat', temp_df.newcat.getItem('budget_cat')).withColumn('ratings', temp_df.newcat.getItem('ratings')).drop('newcat')
df_with_newcols.show(15,False)

+-----+---------+----------+----------+-------+
|id   |budget   |popularity|budget_cat|ratings|
+-----+---------+----------+----------+-------+
|43000|0.0      |2.503     |Small     |Low    |
|43001|0.0      |5.51      |Small     |High   |
|43002|0.0      |5.62      |Small     |High   |
|43003|0.0      |7.159     |Small     |High   |
|43004|500000.0 |3.988     |Small     |Mid    |
|43006|0.0      |3.194     |Small     |Mid    |
|43007|0.0      |2.689     |Small     |Low    |
|43008|0.0      |6.537     |Small     |High   |
|43010|0.0      |4.297     |Small     |Mid    |
|43011|0.0      |4.417     |Small     |Mid    |
|43012|7000000.0|4.722     |Small     |Mid    |
|43013|0.0      |2.543     |Small     |Low    |
|43014|0.0      |4.303     |Small     |Mid    |
|43015|0.0      |3.493     |Small     |Mid    |
|43016|0.0      |2.851     |Small     |Low    |
+-----+---------+----------+----------+-------+
only showing top 15 rows



In [43]:
#  New Columns - Observe Metadata
temp_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- budget: float (nullable = true)
 |-- popularity: float (nullable = true)
 |-- newcat: struct (nullable = true)
 |    |-- budget_cat: string (nullable = true)
 |    |-- ratings: string (nullable = true)



In [44]:
#  New Columns - Method 3
df_with_newcols = df.select('id','budget','popularity').\
withColumn('budget_cat', when(df['budget']<10000000,'Small').when(df['budget']<100000000,'Medium').otherwise('Big')).\
withColumn('ratings', when(df['popularity']<3,'Low').when(df['popularity']<5,'Mid').otherwise('High'))

In [45]:
#  Dropping and renaming
columns_to_drop=['budget_cat']
df_with_newcols=df_with_newcols.drop(*columns_to_drop)
df_with_newcols = df_with_newcols.withColumnRenamed('id','film_id') .withColumnRenamed('ratings','film_ratings')
# You can define all the variable changes in the list
new_names = [('budget','film_budget'),('popularity','film_popularity')]

# Applying the alias function
df_with_newcols_renamed = df_with_newcols.select(list(map(lambda old,new:col(old).alias(new),*zip(*new_names))))

In [46]:
#  Observe data
df_with_newcols.show()

+-------+---------+----------+------------+
|film_id|   budget|popularity|film_ratings|
+-------+---------+----------+------------+
|  43000|      0.0|     2.503|         Low|
|  43001|      0.0|      5.51|        High|
|  43002|      0.0|      5.62|        High|
|  43003|      0.0|     7.159|        High|
|  43004| 500000.0|     3.988|         Mid|
|  43006|      0.0|     3.194|         Mid|
|  43007|      0.0|     2.689|         Low|
|  43008|      0.0|     6.537|        High|
|  43010|      0.0|     4.297|         Mid|
|  43011|      0.0|     4.417|         Mid|
|  43012|7000000.0|     4.722|         Mid|
|  43013|      0.0|     2.543|         Low|
|  43014|      0.0|     4.303|         Mid|
|  43015|      0.0|     3.493|         Mid|
|  43016|      0.0|     2.851|         Low|
|  43017|      0.0|     4.047|         Mid|
|  43018|      0.0|     2.661|         Low|
|  43019|      0.0|     3.225|         Mid|
|  43020|      0.0|      5.72|        High|
|  43021|      0.0|     3.292|  